In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import ipypivot as pt

# Demo notebook for `ipypivot`
+ This [Jupyter widget](https://ipywidgets.readthedocs.io/en/stable/#) wraps the very convenient [pivotTable.js lib](https://pivottable.js.org/examples/)
+ The examples below are reproduced from the pivotTable.js [examples page](https://pivottable.js.org/examples/)
+ `.Pivot()` corresponds to pivotTable.js [`pivot()` method](https://github.com/nicolaskruchten/pivottable/wiki/Parameters#pivotinput-options-locale)
+ `.PivotUI_Box()` corresponds to pivotTable.js [`pivotUI()` method](https://github.com/nicolaskruchten/pivottable/wiki/Parameters#pivotuiinput-options-overwrite-locale)
+ data must be input as a [pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)
+ options must be input as a `Pivot_Options` or `PivotUI_Options` object.
    + key-values as per the pivotTable.js [documentation](https://github.com/nicolaskruchten/pivottable/wiki/Parameters)
    + These objects have first level autocomplete.
+ A `PivotUI` object p (say `p` for example) has 2 buttons: **Save** and **Restore**.
    + **Save** to snapshot the current configuration into dataframe `p.table.df_export` and dict `p.table.options`.  
    _Note_: Options that are javascript functions are discarded when passed to Python
    + **Restore** to apply the options saved
+ Bidirectional synchronization is applied (see the end of the notebook for examples)
    + from JS to Python: as explained above, if you change the table configuration from the JS and hit the **Save** button, it will be saved in the Python side.
    + from Python to JS: if any attribute of the `option` object is changed from the Python side, all views will be automatically re-rerendered.

# Examples of pivot tables
## 1 - pivot - Base
+ Cf. original in [jsfiddle](https://jsfiddle.net/nicolaskruchten/kn381h7s/)

In [ ]:
df = pt.samples.df_tips
p1 = pt.Pivot(df_data=df)
opts = p1.options
opts.rows = ['sex', 'smoker']
opts.cols = ['day', 'time']
opts.aggregatorName = 'Sum over Sum'
opts.vals = ["tip", "total_bill"]
opts.rendererName = 'Table Heatmap'

p1

In [ ]:
p1.options

## 2 - pivotUI - Base
+ Options as `PivotUI_Options` helper object
+ Cf. [original jsfiddle](https://jsfiddle.net/nicolaskruchten/kn381h7s/)

In [ ]:
df = pt.samples.df_tips

p2 = pt.PivotUI(df_data=df)
opts = p2.table.options

opts.rows = ['sex', 'smoker']
opts.cols = ['day', 'time']
opts.vals = ['tip', 'total_bill']
opts.aggregatorName = 'Sum over Sum'
opts.rendererName = 'Table Heatmap'
p2

In [ ]:
p2.table.options.rows = ['sex', 'smoker']

In [ ]:
p2.table._options

In [ ]:
p2.table.options

In [ ]:
p2.table.df_export

## 3 - pivotUI - derived attributes
+ Cf. original in [pivotTable.js example](https://pivottable.js.org/examples/mps.html)

In [ ]:
df = pt.samples.df_mps

p3 = pt.PivotUI(df_data=df)
opts = p3.table.options

opts.rows = ['Gender Imbalance']
opts.cols = ['Age Bin']
opts.derivedAttributes = {
    'Age Bin': 'Utilities.derivers.bin("Age", 10)',
    'Gender Imbalance': 'function(mp) { return mp["Gender"] == "Male" ? 1 : -1; }'    
}
opts.unusedOrientationCutoff = 100000

p3

## 5 - pivotUI - prepopulated with click callback
+ Cf. [original pivotTable.js example](https://pivottable.js.org/examples/mps_prepop.html)

In [ ]:
df = pt.samples.df_mps

p5 = pt.PivotUI(df_data=df)
opts = p5.table.options

opts.rows = ['Province']
opts.cols = ['Party']
opts.aggregatorName = 'Integer Sum'
opts.vals = ['Age']
opts.rendererName = 'Table Heatmap'
opts.rendererOptions = {
    'table': {
        'clickCallback': """function(e, value, filters, pivotData){
                            var names = [];
                            pivotData.forEachMatchingRecord(filters,
                                function(record){ names.push(record.Name); });
                            alert(names.join(\"\\n\"));
                        }"""
    }
}

print('-> Click on a cell to view the contents')  ### does not work for now...
p5

In [ ]:
p5.table.options

## 6 - pivotUI - custom aggregators and sort order
+ Cf. [original pivotTable.js example](https://pivottable.js.org/examples/mps_agg.html)

In [ ]:
df = pt.samples.df_mps

p6 = pt.PivotUI(df_data=df)
opts = p6.table.options

dic = {
    'tpl': 'Utilities.aggregatorTemplates',
    'sortAs': 'Utilities.sortAs'
}

opts.rows = ['Province']
opts.cols = ['Party']
opts.aggregators = {
    "Number of MPs":      'function() {{ return {tpl}.count()() }}'.format(**dic),
    "Average Age of MPs": 'function() {{ return {tpl}.average()(["Age"]) }}'.format(**dic)
}
opts.aggregatorName = 'Number of MPs'
opts.sorters = {
    'Age': 'function(a,b){ return b-a; }', # sort backwards
    'Province': """{sortAs}(["British Columbia", "Alberta", "Saskatchewan", "Manitoba",
                             "Territories", "Ontario", "Quebec", "New Brunswick",
                             "Prince Edward Island", "Nova Scotia",
                             "Newfoundland and Labrador"])""".format(**dic)
}

p6

## 7 - pivotUI - C3 chart renderer
+ Cf. original in [pivotTable.js example](https://pivottable.js.org/examples/c3.html)

In [ ]:
df = pt.samples.df_mps

p7 = pt.PivotUI(df_data=df)
opts = p7.table.options

opts.rows = ['Province']
opts.cols = ['Party']
opts.rendererName = 'Stacked Bar Chart'
opts.rowOrder = 'value_a_to_z'
opts.colOrder = 'value_z_to_a'


p7

In [ ]:
p7.table.options

# Below: To Be Changed to another example

# Examples of bidirectional synchronization
### Saved dataframe

+ Snapshot by Save button (first click a few milliseconds post creation)
+ Multi-index corresponding to pivot table


In [ ]:
# p.table.df_export

### Modify options properties

+ Triggers re-rendering of all views

In [ ]:
#p.table.options.aggregatorName = 'Max Temperature'

In [ ]:
#p.table.options.aggregators = {
#    'Mean Temperature': "function () {{ return {tpl}.average({numberFormat})(['Mean Temp (C)']) }}".format(**dic),
#    'Max Temperature': "function () {{ return {tpl}.max({numberFormat})(['Max Temp (C)']) }}".format(**dic)
#}

In [ ]:
#p.table.options.rendererOptions = {
#    'heatmap': {
#        'colorScaleGenerator': """function (values) {
#            return d3.scale.linear()
#                .domain([-20, 0, 20])
#                .range(['#77F', '#FFF', '#F77'])
#        }"""
#    }
#}

In [ ]:
#p.table.options.to_dict()